In [1]:
import pickle

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, roc_curve, auc,ConfusionMatrixDisplay

In [4]:
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
import xgboost as xgb

In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
#from sklearn.pipeline import make_pipeline

In [6]:
from imblearn.pipeline import make_pipeline

In [7]:
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier

In [8]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

In [9]:
from sklearn.base import TransformerMixin

In [10]:
from imblearn.over_sampling import SMOTE,RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import ADASYN
from imblearn.under_sampling import TomekLinks

In [11]:
from sklearn.model_selection import GridSearchCV

In [12]:
pkl_file = open('X_train.pkl', 'rb')
X_train = pickle.load(pkl_file)
pkl_file.close()

In [13]:
pkl_file = open('X_test.pkl', 'rb')
X_test = pickle.load(pkl_file)
pkl_file.close()

In [14]:
pkl_file = open('y_train.pkl', 'rb')
y_train = pickle.load(pkl_file)
pkl_file.close()

In [15]:
pkl_file = open('y_test.pkl', 'rb')
y_test = pickle.load(pkl_file)
pkl_file.close()

In [16]:
df=pd.read_csv('../Capstone-3-/data.csv')

In [17]:
df

,Id,Income,Age,Experience,Marital_Status,House_Ownership,Car_Ownership,Profession,Job_Yrs,House_Yrs,Risk_Flag
0,1,1303834,23,3,SINGLE,RENTED,NO,MECHANICAL_ENGINEER,3,13,0
1,2,7574516,40,10,SINGLE,RENTED,NO,SOFTWARE_DEVELOPER,9,13,0
2,3,3991815,66,4,MARRIED,RENTED,NO,TECHNICAL_WRITER,4,10,0
3,4,6256451,41,2,SINGLE,RENTED,YES,SOFTWARE_DEVELOPER,2,12,1
4,5,5768871,47,11,SINGLE,RENTED,NO,CIVIL_SERVANT,3,14,1
...,...,...,...,...,...,...,...,...,...,...,...
251995,251996,8154883,43,13,SINGLE,RENTED,NO,SURGEON,6,11,0
251996,251997,2843572,26,10,SINGLE,RENTED,NO,ARMY_OFFICER,6,11,0
251997,251998,4522448,46,7,SINGLE,RENTED,NO,DESIGN_ENGINEER,7,12,0
251998,251999,6507128,45,0,SINGLE,RENTED,NO,GRAPHIC_DESIGNER,0,10,0


In [18]:
X_train.shape

(176400, 63)

In [19]:
y_train.shape 

(176400, 1)

In [20]:
resamplers={}
resamplers['SMOTE']=SMOTE()
resamplers['AD']=ADASYN()
resamplers['Random under Sampler']=RandomUnderSampler()
resamplers['Random over Sampler']=RandomOverSampler()
resamplers['TomekLinks']=TomekLinks()

In [21]:
classifiers = {}
classifiers['LR'] = LogisticRegression()
classifiers['DT'] = DecisionTreeClassifier()
classifiers['RF'] = RandomForestClassifier()
classifiers['KNN'] = KNeighborsClassifier()
classifiers['LGBM'] = LGBMClassifier()
classifiers['XGB'] = xgb.XGBClassifier(objective="binary:logistic", use_label_encoder=False, eval_metric='logloss')

In [22]:
scores = {}
cls_report= {}
scores['Accuracy'] = pd.DataFrame(columns=classifiers.keys(), index=resamplers.keys())
scores['AUC'] = pd.DataFrame(columns=classifiers.keys(), index=resamplers.keys())

In [96]:
#class DenseTransformer(TransformerMixin):
#    def fit(self,X,y=None,**fit_params):
#        return self
#    def transform(self,X,y=None, **fit_params):
#        return X.todense()

In [98]:
for model in classifiers.keys():
    for resampler in resamplers.keys():
        pipe = make_pipeline(resamplers[resampler], classifiers[model])
        pipe.fit(X_train, y_train)
        scores['Accuracy'].loc[resampler,model] = pipe.score(X_test, y_test)
        scores['AUC'].loc[resampler,model] = roc_auc_score(y_test, pipe.predict(X_test))
        cls_report[resampler,model]=classification_report(y_test, pipe.predict(X_test))

In [20]:
scores['Accuracy'].mean()

NameError: name 'scores' is not defined

In [102]:
scores['AUC']

,LR,DT,RF,KNN,LGBM,XGB
SMOTE,0.840881,0.849951,0.839172,0.849277,0.754516,0.647893
AD,0.841741,0.847941,0.839726,0.846804,0.717778,0.600201
Random under Sampler,0.841844,0.85327,0.845875,0.828299,0.725941,0.694265
Random over Sampler,0.840259,0.850401,0.840688,0.855157,0.750849,0.696978
TomekLinks,0.708762,0.740363,0.747333,0.725811,0.501937,0.5


In [100]:
for key, value in cls_report.items():
...     print(key,'\n', value)

('SMOTE', 'LR') 
               precision    recall  f1-score   support

           0       0.97      0.91      0.94     66301
           1       0.55      0.77      0.64      9299

    accuracy                           0.90     75600
   macro avg       0.76      0.84      0.79     75600
weighted avg       0.92      0.90      0.90     75600

('AD', 'LR') 
               precision    recall  f1-score   support

           0       0.97      0.91      0.94     66301
           1       0.55      0.77      0.64      9299

    accuracy                           0.89     75600
   macro avg       0.76      0.84      0.79     75600
weighted avg       0.91      0.89      0.90     75600

('Random under Sampler', 'LR') 
               precision    recall  f1-score   support

           0       0.97      0.91      0.94     66301
           1       0.54      0.78      0.64      9299

    accuracy                           0.89     75600
   macro avg       0.75      0.84      0.79     75600
weighted

SMTOE will be used for parameter tuning 

In [20]:
sm=SMOTE()
X_train_sm,y_train_sm=sm.fit_resample(X_train,y_train)

In [56]:
DTT=DecisionTreeClassifier()

In [57]:
DTT.fit(X_train_sm, y_train_sm)

DecisionTreeClassifier()

In [58]:
DTT.score(X_test, y_test)

0.8803968253968254

In [43]:
DTT.get_params

<bound method BaseEstimator.get_params of RandomForestClassifier(criterion='entropy', max_depth=8, max_features=7)>

In [59]:
cls=classification_report(y_test, DTT.predict(X_test))

In [60]:
print(cls)

              precision    recall  f1-score   support

           0       0.97      0.89      0.93     66301
           1       0.51      0.81      0.62      9299

    accuracy                           0.88     75600
   macro avg       0.74      0.85      0.78     75600
weighted avg       0.91      0.88      0.89     75600



In [ ]:
# choose a resampling mehtod for param tuning using below and print out scores, classificaition report, auc roc, 

In [ ]:
names = [
        "Logistic Regression",
         "DecisionTreeClassifier",
         "RandomForestClassifier",
         'KNeighborsClassifier',
         "LGBMClassifier",
         "XGBClassifier"
        ]

classifiers = [
    LogisticRegression(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    KNeighborsClassifier(),
    LGBMClassifier(),
    xgb.XGBClassifier(objective="binary:logistic", use_label_encoder=False, eval_metric='logloss')
    
]

parameters = [
              {"clf__penalty": ['l2','l1'],
               "clf__C": np.logspace(0,1, 4, 10)}, #logistic regression
              {'clf__max_depth': [2, 3, 5, 10],
               'clf__min_samples_leaf': [1,5, 10, 20, 50], 
               'clf__criterion': ["gini", "entropy"]},# decision trees
              {'clf__n_estimators':[100,1000],
               'clf__max_depth':[0,4,5,6,8],
               'clf__criterion':['gini','entropy'],
               'clf__max_features':['sqrt', 'log2', None],
               'clf__min_samples_split':[1,2,3]}, #Random Forest
             {'clf__n_neighbors' : [5,7,9,11,13,15],
               'clf__weights' : ['uniform','distance'],
               'clf__metric' : ['minkowski','euclidean','manhattan']},#KNN
              {'clf__learning_rate': [0.1,0.05,0.01,0.0001],
               'clf__num_leaves': [31,90,140,200],
               'clf__boosting_type' : ['gbdt'],
               'clf__objective' : [None,'binary'],
               'clf__max_depth' : [-1,4,5,6,7]},#LGBM
              {'clf__max_depth': [3, 5, 6, 10],
               'clf__learning_rate': [0.1, 0.2, 0.3],
               'clf__subsample': np.arange(0.6, 0.8, 1.0),
              'clf__n_estimators': [100, 500, 1000],}#xgb
              ]

for name, classifier, params in zip(names, classifiers, parameters):
    clf_pipe = Pipeline([('clf', classifier)])
    gs_clf = GridSearchCV(clf_pipe, param_grid=params, n_jobs=None,scoring='recall')
    clf = gs_clf.fit(X_train_sm, y_train_sm)
    score = clf.score(X_test, y_test)
    best_param=clf.best_params_
    roc_score=roc_auc_score(y_test, gs_clf.predict(X_test))
    cls_report=classification_report(y_test, gs_clf.predict(X_test))
    print("{} accuracy score: {}".format(name, score))
    print("{} best parameters: {}".format(name,best_param))
    print("{} roc_auc: {}".format(name,roc_score))
    print("{} confusion matrix: {}".format(name,cls_report))

Logistic Regression accuracy score: 0.4411227013657383
Logistic Regression best parameters: {'clf__C': 2.154434690031884, 'clf__penalty': 'l2'}
Logistic Regression roc_auc: 0.5430904226425681
Logistic Regression confusion matrix:               precision    recall  f1-score   support

           0       0.89      0.65      0.75     66301
           1       0.15      0.44      0.22      9299

    accuracy                           0.62     75600
   macro avg       0.52      0.54      0.49     75600
weighted avg       0.80      0.62      0.68     75600

DecisionTreeClassifier accuracy score: 0.8537477148080439
DecisionTreeClassifier best parameters: {'clf__criterion': 'gini', 'clf__max_depth': 5, 'clf__min_samples_leaf': 1}
DecisionTreeClassifier roc_auc: 0.5313594609394136
DecisionTreeClassifier confusion matrix:               precision    recall  f1-score   support

           0       0.91      0.21      0.34     66301
           1       0.13      0.85      0.23      9299

    accuracy 

In [ ]:
names = [
        #"Logistic Regression",
         #"DecisionTreeClassifier",
         #"RandomForestClassifier",
         'KNeighborsClassifier',
         "LGBMClassifier",
         "XGBClassifier"
        ]

classifiers = [
    #LogisticRegression(),
    #DecisionTreeClassifier(),
    #RandomForestClassifier(),
    KNeighborsClassifier(),
    LGBMClassifier(),
    xgb.XGBClassifier(objective="binary:logistic", use_label_encoder=False, eval_metric='logloss')
    
]

parameters = [
              #{"clf__penalty": ['l2','l1'],
               #"clf__C": np.logspace(0,1, 4, 10)}, #logistic regression
              #{'clf__max_depth': [2, 3, 5, 10],
              # 'clf__min_samples_leaf': [1,5, 10, 20, 50], 
               #'clf__criterion': ["gini", "entropy"]},# decision trees
              #{'clf__n_estimators':[100,1000],
               #'clf__max_depth':[0,4,5,6,8],
               #'clf__criterion':['gini','entropy'],
               #'clf__max_features':['sqrt', 'log2', None],
               #'clf__min_samples_split':[1,2,3]}, #Random Forest
             {'clf__n_neighbors' : [5,9,13],
               'clf__weights' : ['uniform','distance'],
               'clf__metric' : ['minkowski','euclidean','manhattan']},#KNN
              {'clf__learning_rate': [0.1,0.01,0.0001],
               'clf__num_leaves': [31,140,200],
               'clf__boosting_type' : ['gbdt'],
               'clf__objective' : [None,'binary'],
               'clf__max_depth' : [-1,5,7]},#LGBM
              {'clf__max_depth': [3, 6, 10],
               'clf__learning_rate': [0.1, 0.2, 0.3],
               'clf__subsample': np.arange(0.6, 0.8, 1.0),
              'clf__n_estimators': [100, 500, 1000],}#xgb
              ]

for name, classifier, params in zip(names, classifiers, parameters):
    clf_pipe = Pipeline([('clf', classifier)])
    gs_clf = GridSearchCV(clf_pipe, param_grid=params, n_jobs=None,scoring='recall')
    clf = gs_clf.fit(X_train_sm, y_train_sm)
    score = clf.score(X_test, y_test)
    best_param=clf.best_params_
    roc_score=roc_auc_score(y_test, gs_clf.predict(X_test))
    cls_report=classification_report(y_test, gs_clf.predict(X_test))
    print("{} accuracy score: {}".format(name, score))
    print("{} best parameters: {}".format(name,best_param))
    print("{} roc_auc: {}".format(name,roc_score))
    print("{} confusion matrix: {}".format(name,cls_report))

hyper parameter tuning with best estimators and confusion matrix, auc

In [28]:
lr_p= {'n_estimators':np.arange(1,10),
               'max_depth':[4,5],
               'criterion':['gini'],
               'max_features':[3]}

In [33]:
RF=RandomForestClassifier(random_state=2)
rf_grid=GridSearchCV(estimator=RF,param_grid=p,cv=3,n_jobs=-1,verbose=2,scoring="recall")
rf_grid.fit(X_train_sm,y_train_sm)
y_pred_rf=rf_grid.predict(X_test)
print(confusion_matrix(y_test,y_pred_rf))
rf_best_par=rf_grid.best_params_

Fitting 3 folds for each of 18 candidates, totalling 54 fits
[[    0 66301]
 [    0  9299]]


In [35]:
rf_best_par

{'criterion': 'gini', 'max_depth': 4, 'max_features': 3, 'n_estimators': 1}

In [36]:
RF=RandomForestClassifier(criterion= 'gini', max_depth= 4, max_features= 3, n_estimators=1)
RF.fit(X_train_sm,y_train_sm)
ypred=RF.predict(X_test)

In [ ]:
#interpretability 

Feature Importance


Feature importance Logistic Regression

In [ ]:
(lr_grid.coef_)
# Those values, however, will show that the second parameter
# is more influential
print(np.std(X_train_sm, 0)*m.coef_)

In [ ]:
feature_importance = abs(lr_grid.coef_[0])
feature_importance = 100.0 * (feature_importance / feature_importance.max())
sorted_idx = np.argsort(feature_importance)
pos = np.arange(sorted_idx.shape[0]) + .5

featfig = plt.figure()
featax = featfig.add_subplot(1, 1, 1)
featax.barh(pos, feature_importance[sorted_idx], align='center')
featax.set_yticks(pos)
featax.set_yticklabels(np.array(X.columns)[sorted_idx], fontsize=8)
featax.set_xlabel('Relative Feature Importance')

plt.tight_layout()   
plt.show()

In [37]:
importances=RF.feature_importances_
importances

array([0., 0., 0., ..., 0., 0., 0.])

In [38]:
std = np.std([tree.feature_importances_ for tree in RF.estimators_], axis=0)

In [41]:
X_train_sm

<309406x31396 sparse matrix of type '<class 'numpy.float64'>'
	with 4436209 stored elements in Compressed Sparse Row format>

In [39]:
std1=pd.Series(std,index=X_train_sm.columns)
std1.sort_values(ascending=False)
std1=std1.head(9)

AttributeError: columns not found

In [ ]:
forest_importances = pd.Series(importances, index=X_train_sm.columns)

In [ ]:
forest_importances1=forest_importances.sort_values(ascending=False).head(9)

In [ ]:
fig, ax = plt.subplots()
forest_importances1.plot.bar(yerr=std1, ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()

In [45]:
df_x = df.loc[:, df.columns != 'Risk_Flag']
df_y=df.loc[:,df.columns == 'Risk_Flag']

In [46]:
X_train1,X_test1,y_train1,y_test1=train_test_split(df_x,df_y,test_size=0.3,random_state=42,stratify=df_y)

In [48]:
std_sc=StandardScaler()

In [56]:
X_train1=X_train1.set_index('Id')

In [57]:
aaaaa=OneHotEncoder()

In [60]:
aaaaa.get_feature_names_out()

array(['Income_10310', 'Income_10675', 'Income_11114', ...,
       'House_Yrs_12', 'House_Yrs_13', 'House_Yrs_14'], dtype=object)

In [59]:
aaaaa.fit_transform(X_train1)

<176400x31391 sparse matrix of type '<class 'numpy.float64'>'
	with 1587600 stored elements in Compressed Sparse Row format>